In [2]:
import cudf
import numpy as np
import os 
import shutil
import pandas as pd 
from scipy import stats
from scipy.stats import chi2_contingency
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
# I use rapids 21-12 in Hipergator

# 1. Get array of population IDs

## PD or HC or subset of PD (prevalent, or incident)

# 1.1 Overall PD 

In [2]:
#PD subjects guaranteede
subject_ids = ["1114277",
"1178636",
"1227372",
"1293346",
"1328327",
"1344871",
"1385201",
"1448986",
"1604568",
"1683568",
"1686653",
"1702601",
"1729129",
"1779711",
"1851133",
"2045185",
"2063006",
"2167017",
"2234655",
"2253228",
"2259013",
"2311988",
"2517130",
"2583580",
"2791680",
"2890808",
"2932050",
"2984777",
"3033844",
"3134693",
"3156084",
"3198287",
"3382506",
"3383380",
"3391006",
"3554033",
"3561789",
"3568167",
"3598466",
"3706056",
"3756652",
"3802938",
"3826232",
"4011374",
"4038762",
"4069794",
"4127426",
"4130938",
"4215662",
"4244556",
"4322638",
"4375090",
"4415530",
"4472792",
"4530859",
"4532453",
"4535648",
"4548991",
"4614800",
"4619705",
"4738821",
"4751589",
"4798230",
"4868455",
"4907629",
"5035719",
"5101325",
"5157987",
"5208752",
"5221906",
"5229530",
"5424494",
"5471094",
"5527470",
"5563399",
"5572442",
"5640521",
"5651883",
"5659619",
"5749875",
"5776153",
"5841706",
"5861534",
"5953087"]

print(len(subject_ids))

84


# 1.2 HC 

In [5]:


data_path = '/blue/ruogu.fang/charlietran/PD_Reproduction_V4/data/Raw_Data/PD_prevalent/'

subject_ids =[]

for file_path in os.listdir(data_path):
    subject_ids.append(file_path[0:7])
subject_ids = np.unique(subject_ids)

print(len(subject_ids), subject_ids)

91


# 1.3 Prevalent PD

In [5]:
data_path = '/blue/ruogu.fang/charlietran/PD_Reproduction_V4/data/Raw_Data/PD_prevalent/'

subject_ids =[]

for file_path in os.listdir(data_path):
    subject_ids.append(file_path[0:7])
subject_ids = np.unique(subject_ids)

print(len(subject_ids), subject_ids)

55 ['1178636' '1227372' '1448986' '1604568' '1683568' '1702601' '1779711'
 '1851133' '2045185' '2234655' '2253228' '2259013' '2311988' '2517130'
 '2791680' '2932050' '2984777' '3033844' '3134693' '3156084' '3383380'
 '3554033' '3561789' '3568167' '3706056' '3756652' '3802938' '3826232'
 '4011374' '4038762' '4127426' '4130938' '4415530' '4472792' '4530859'
 '4535648' '4548991' '4619705' '4738821' '4798230' '4907629' '5035719'
 '5157987' '5208752' '5221906' '5229530' '5471094' '5527470' '5640521'
 '5651883' '5749875' '5776153' '5841706' '5861534' '5953087']


# 1.4 Incident PD 

In [6]:


data_path = '/blue/ruogu.fang/charlietran/PD_Reproduction_V4/data/Raw_Data/PD_incident/'

subject_ids =[]

for file_path in os.listdir(data_path):
    subject_ids.append(file_path[0:7])
subject_ids = np.unique(subject_ids)

print(len(subject_ids), subject_ids)

29 ['1114277' '1293346' '1328327' '1344871' '1385201' '1686653' '1729129'
 '2063006' '2167017' '2583580' '2890808' '3198287' '3382506' '3391006'
 '3598466' '4069794' '4215662' '4244556' '4322638' '4375090' '4532453'
 '4614800' '4751589' '4868455' '5101325' '5424494' '5563399' '5572442'
 '5659619']


# 2. Use RAPIDS Cudf to read csv and load data statistics

In [3]:
# 


csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb669006.csv'

df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]


print('------------------------------------------------------------------------------')
print('Townsend Indices')
print('Mean of Townsend Indices', df4['189-0.0'].astype('float64').mean())
print('Standard Deviation of Townsend Indices', df4['189-0.0'].astype('float64').std())
print('------------------------------------------------------------------------------')
print('Stroke History')
print('Number of Stroke Subjects', df4['42006-0.0'].notnull().sum())
print('Number of Non-Stroke Subjects', df4['42006-0.0'].isnull().sum())
print('------------------------------------------------------------------------------')
print('Smoking History')
print('Number of Smoking Subjects', (df4['20116-0.0'].astype('uint8') != 0).sum())
print('Number of Non-Smoking Subjects', (df4['20116-0.0'].astype('uint8') == 0).sum())
print('------------------------------------------------------------------------------')
print('Alcohol Status')
print('Number of Alcohol Subjects', (df4['20117-0.0'].astype('uint8') != 0).sum())
print('Number of Non-Alcohol Subjects', (df4['20117-0.0'].astype('uint8') == 0).sum())
print('------------------------------------------------------------------------------')
print('Obesity-Diabetes Status')
print('Number of Obesity Subjects', (df4['21001-0.0'].astype('float64') > 30).sum())
print('Number of Non-Obesity Subjects', (df4['21001-0.0'].astype('float64') < 30).sum())
print('------------------------------------------------------------------------------')
print('Psychotropic Medication')
print('Number of Medication Subjects', (df4['20466-0.0'].astype('uint8') == 1).sum())
print('Number of Non-Medication Subjects', (df4['20466-0.0'].astype('uint8') != 1).sum() 
      + df4['20466-0.0'].astype('uint8').isnull().sum())

#df4.to_csv('/blue/ruogu.fang/charlietran/UKB/csv/PD_characteristics.csv')


------------------------------------------------------------------------------
Townsend Indices
Mean of Townsend Indices -1.4788360714285713
Standard Deviation of Townsend Indices 2.947206113955094
------------------------------------------------------------------------------
Stroke History
Number of Stroke Subjects 3
Number of Non-Stroke Subjects 81
------------------------------------------------------------------------------
Smoking History
Number of Smoking Subjects 30
Number of Non-Smoking Subjects 54
------------------------------------------------------------------------------
Alcohol Status
Number of Alcohol Subjects 80
Number of Non-Alcohol Subjects 4
------------------------------------------------------------------------------
Obesity-Diabetes Status
Number of Obesity Subjects 18
Number of Non-Obesity Subjects 66
------------------------------------------------------------------------------
Psychotropic Medication
Number of Medication Subjects 0
Number of Non-Medication Subj

In [11]:
csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb52184.csv'

df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

A = df4[['eid', '21003-0.0', '31-0.0', '21000-0.0', '5206-0.0', '5199-0.0',]]
A['sum_total'] = A['5206-0.0'].astype('float64') + A['5199-0.0'].astype('float64')

print('Mean Age:', A['21003-0.0'].astype('float64').mean())
print('Stdev Age:', A['21003-0.0'].astype('float64').std())
print('----------------------------------------------------------------')
print('Number of Males', (A['31-0.0'].astype('float64') == 1).sum(), '| Percentage ', (A['31-0.0'].astype('float64') == 1).sum() / len(df4))
print('Number of Females', (A['31-0.0'].astype('float64') == 0).sum(), '| Percentage ', (A['31-0.0'].astype('float64') == 0).sum() / len(df4))
print('----------------------------------------------------------------')
print('Number of White Ethnicity',  (A['21000-0.0'].astype('int32') == 1001).sum() + (A['21000-0.0'].astype('int32') == 1002).sum(),
     ' | Percentage ', 100* ((A['21000-0.0'].astype('int32') == 1001).sum() + (A['21000-0.0'].astype('int32') == 1002).sum()) / len(df4) 
     )
print('Number of White Ethnicity',  len(df4) - ((A['21000-0.0'].astype('int32') == 1001).sum() + (A['21000-0.0'].astype('int32') == 1002).sum()),
     ' | Percentage ', 100* (len(df4) - ((A['21000-0.0'].astype('int32') == 1001).sum() + (A['21000-0.0'].astype('int32') == 1002).sum())) / len(df4)
     )
print('----------------------------------------------------------------')
print('Mean Visual Acuity',  A['sum_total'].mean())
print('Std Visual Acuity',  A['sum_total'].std())

Mean Age: 61.82142857142857
Stdev Age: 5.929628920886516
----------------------------------------------------------------
Number of Males 49 | Percentage  0.5833333333333334
Number of Females 35 | Percentage  0.4166666666666667
----------------------------------------------------------------
Number of White Ethnicity 82  | Percentage  97.61904761904762
Number of White Ethnicity 2.0  | Percentage  2.380952380952381
----------------------------------------------------------------
Mean Visual Acuity 0.28400000000000003
Std Visual Acuity 0.3834551314532611


In [12]:
csv_dir = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52152.csv'
df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

A = df4[['6148-0.0']]

# healthy accounts is

print('Number of Vision Problem Subjects', (len(df4) - (A['6148-0.0'].astype('float64').isin([-3, -7, -1]).sum() 
                                            + A['6148-0.0'].astype('float64').isnull().sum())),
     ' | Percentage ', 100 * (len(df4) - (A['6148-0.0'].astype('float64').isin([-3, -7, -1]).sum() 
                                            + A['6148-0.0'].astype('float64').isnull().sum())) / len(df4))

print('Number of Healthy Vision Subjects', (A['6148-0.0'].astype('float64').isin([-3, -7, -1]).sum() 
                                            + A['6148-0.0'].astype('float64').isnull().sum()),
     ' | Percentage ', 100 * (A['6148-0.0'].astype('float64').isin([-3, -7, -1]).sum() 
                                            + A['6148-0.0'].astype('float64').isnull().sum()) / len(df4))

Number of Vision Problem Subjects 9.0  | Percentage  10.714285714285714
Number of Healthy Vision Subjects 75  | Percentage  89.28571428571429


In [14]:
from scipy.stats import chi2_contingency

yes = np.array([1, 2])
no = np.array([52, 29])

obs = obs = np.vstack([yes, no])
chi, p, _, expected = chi2_contingency(obs, correction = False)
print('pvalue', p)
expected
#_, p _, _ = chi2_contingency(true_obs, false_obs)
#p


pvalue 0.2766503035738431


array([[ 1.89285714,  1.10714286],
       [51.10714286, 29.89285714]])

# Dates of Diagnoses and Visit Dates

In [54]:
csv_dir = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52164.csv'
df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

df4 = df4[['eid', '53-0.0']]

csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'
df5 = cudf.read_csv(csv_dir)
df5 = df5.loc[df5['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

#df4[['53-0.0', '42032-0.0']]
df5 = df5[['eid', '42032-0.0']]

df6 = cudf.merge(df4, df5, on ="eid")
df6 = df6.sort_values(by = ['eid'])

#df6.to_csv('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/results/Prevalent_PD_Dates.csv')

In [55]:
df5

,eid,42032-0.0
8320,1956515,<NA>
9061,5810422,<NA>
16983,2950751,<NA>
27317,1300508,<NA>
45693,5208893,<NA>
...,...,...
461524,2873857,<NA>
462259,2341415,<NA>
474070,1855467,<NA>
476662,5602865,<NA>


In [55]:
csv_dir = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52164.csv'
df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

df4 = df4[['eid', '53-0.0', '21003-0.0', '21003-1.0']]

csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'
df5 = cudf.read_csv(csv_dir)
df5 = df5.loc[df5['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

#df4[['53-0.0', '42032-0.0']]
df5 = df5[['eid', '42030-0.0']]

df6 = cudf.merge(df4, df5, on ="eid")
df6 = df6.sort_values(by = ['eid'])

df6.to_csv('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/results/Incident_PD_Dates.csv')

In [3]:
# OVERALL PARKINSON"S DISEASE
subject_ids = ["1114277",
"1178636",
"1227372",
"1293346",
"1328327",
"1344871",
"1385201",
"1448986",
"1604568",
"1683568",
"1686653",
"1702601",
"1729129",
"1779711",
"1851133",
"2045185",
"2063006",
"2167017",
"2234655",
"2253228",
"2259013",
"2311988",
"2517130",
"2583580",
"2791680",
"2890808",
"2932050",
"2984777",
"3033844",
"3134693",
"3156084",
"3198287",
"3382506",
"3383380",
"3391006",
"3554033",
"3561789",
"3568167",
"3598466",
"3706056",
"3756652",
"3802938",
"3826232",
"4011374",
"4038762",
"4069794",
"4127426",
"4130938",
"4215662",
"4244556",
"4322638",
"4375090",
"4415530",
"4472792",
"4530859",
"4532453",
"4535648",
"4548991",
"4614800",
"4619705",
"4738821",
"4751589",
"4798230",
"4868455",
"4907629",
"5035719",
"5101325",
"5157987",
"5208752",
"5221906",
"5229530",
"5424494",
"5471094",
"5527470",
"5563399",
"5572442",
"5640521",
"5651883",
"5659619",
"5749875",
"5776153",
"5841706",
"5861534",
"5953087"]


csv_dir = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52164.csv'
df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

df4 = df4[['eid', '53-0.0', '21003-0.0', '21003-1.0']]

csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'
df5 = cudf.read_csv(csv_dir)
df5 = df5.loc[df5['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

#df4[['53-0.0', '42032-0.0']]
df5 = df5[['eid', '42030-0.0']]

df6 = cudf.merge(df4, df5, on ="eid")
df6 = df6.sort_values(by = ['eid'])

df6.to_csv('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/results/OVERALL_PD_Dates.csv')

In [4]:
# OVERALL PARKINSON"S DISEASE
subject_ids = ["1034220",
"1112794",
"1136758",
"1143056",
"1207549",
"1291253",
"1300508",
"1338245",
"1385711",
"1445308",
"1454985",
"1463230",
"1523027",
"1523223",
"1555296",
"1558677",
"1561855",
"1562567",
"1578746",
"1605541",
"1619489",
"1646019",
"1714891",
"1743307",
"1756582",
"1762289",
"1786145",
"1823712",
"1855467",
"1956515",
"1991498",
"1998264",
"2003452",
"2005964",
"2022296",
"2054405",
"2100562",
"2252178",
"2317118",
"2325512",
"2338505",
"2341415",
"2518197",
"2558526",
"2573039",
"2682558",
"2747870",
"2770153",
"2770564",
"2789351",
"2819601",
"2822488",
"2848575",
"2873857",
"2923883",
"2950751",
"3056090",
"3214958",
"3247304",
"3486864",
"3573739",
"3595368",
"3687402",
"3821117",
"3850168",
"3943027",
"3946341",
"3963896",
"4001612",
"4287033",
"4356152",
"4528562",
"4627046",
"4652340",
"4754281",
"4855507",
"4959352",
"5168495",
"5208893",
"5286204",
"5318943",
"5394841",
"5444877",
"5462419",
"5602865",
"5623257",
"5745206",
"5749166",
"5810422",
"5887044",
"5955668"]


csv_dir = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52164.csv'
df4 = cudf.read_csv(csv_dir)
df4 = df4.loc[df4['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

df4 = df4[['eid', '53-0.0', '21003-0.0', '21003-1.0']]

csv_dir = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'
df5 = cudf.read_csv(csv_dir)
df5 = df5.loc[df5['eid'].isin(subject_ids)]

#A = df4[['6148-0.0']]

#df4[['53-0.0', '42032-0.0']]
df5 = df5[['eid', '42030-0.0']]

df6 = cudf.merge(df4, df5, on ="eid")
df6 = df6.sort_values(by = ['eid'])

df6.to_csv('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/results/OVERALL_HC_Dates.csv')

In [ ]:
## Get all information directly from this 

path = '/blue/ruogu.fang/charlietran/PD_Reproduction_V2/data/Raw_Data/PD/
files = os.listdir(path) 
csv_0 = '/blue/ruogu.fang/charlietran/UKB/csv/ukb52164.csv'
csv_1 = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'

for i in range(len(os.listdir(path))
    with open(os.path.join(args.project_dir, 'results', 'PD_characteristics.csv'), 'a') as file:

       
               
        subject_id = 
        writer = csv.writer(file, delimiter=',')
        writer.writerow([
                    
                    
                    ])
    

In [3]:
import csv
import time 
from datetime import date

print('Starting')
path = '/blue/ruogu.fang/charlietran/PD_Reproduction_V2/data/Raw_Data/PD/'
csv_0 = '/blue/ruogu.fang/share/RetinaPD/csv/ukb52164.csv'
csv_1 = '/blue/ruogu.fang/share/RetinaPD/csv/ukb37849.csv'
df4 = cudf.read_csv(csv_0)
df5 = cudf.read_csv(csv_1)
files = sorted(os.listdir(path))

# Start the top line of the csv
with open(os.path.join('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/', 'results', 'PD_characteristics_V2.csv'), 'a') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Image',
                     'ID', 
                     'Age',
                     'Assessment_Date',
                     'Diagnosis_Date',
                     'Delta_Days',
                     'Delta_Months',
                     'Delta_Years',
                     'CATEGORY',
                    ])
for img in files:
    subject_eid = img[0:7]
    with open(os.path.join('/blue/ruogu.fang/charlietran/PD_Reproduction_V2/', 'results', 'PD_characteristics_V2.csv'), 'a') as file:
        df6 = df4.loc[df4['eid'] == subject_eid]
        df6 = df6[['eid', '53-0.0', '53-1.0', '21003-0.0', '21003-1.0']]

        df7 = df5.loc[df5['eid'] == subject_eid]
        df7 = df7[['eid', '42032-0.0']]

        
        if img[8:17] == '21015_0_0' or img[8:17] == '21016_0_0':
            visit_date = df6['53-0.0'].to_numpy().item()
            age = df6['21003-0.0'].to_numpy().item()
            diagnosis_date = df7['42032-0.0'].to_numpy().item()         
    
            newdate1 = time.strptime(visit_date, "%Y-%m-%d")
            newdate2 = time.strptime(diagnosis_date, "%Y-%m-%d")
            
            format_date_visit = date(newdate1.tm_year, newdate1.tm_mon, newdate1.tm_mday)
            format_date_diag = date(newdate2.tm_year, newdate2.tm_mon, newdate2.tm_mday)
            
        
        elif img[8:17] == '21015_1_0' or img[8:17] == '21016_1_0':
            visit_date = df6['53-1.0'].to_numpy().item()
            age = df6['21003-1.0'].to_numpy().item()
            diagnosis_date = df7['42032-0.0'].to_numpy().item()
            newdate1 = time.strptime(visit_date, "%Y-%m-%d")
            newdate2 = time.strptime(diagnosis_date, "%Y-%m-%d")
            
            format_date_visit = date(newdate1.tm_year, newdate1.tm_mon, newdate1.tm_mday)
            format_date_diag = date(newdate2.tm_year, newdate2.tm_mon, newdate2.tm_mday)  
            
        if newdate1 >= newdate2: # If the visit date is after the date of assessment then make this positive, this is prevalent
            delta_days = -(format_date_visit - format_date_diag).days  
            delta_years = -(format_date_visit - format_date_diag).days / 365
            delta_months = delta_years * 12 
            tag = 'PREVALENT'
        elif newdate1 < newdate2: #If the visit date is before the date of assessment (we will call this incident PD) 
            delta_days = -(format_date_visit - format_date_diag).days  
            delta_years = -(format_date_visit - format_date_diag).days / 365
            delta_months = delta_years * 12 
            tag = 'INCIDENT'
        writer = csv.writer(file, delimiter=',')
        writer.writerow([img, 
                         subject_eid, 
                         age,
                         visit_date,
                         diagnosis_date,
                         delta_days,
                         delta_months,
                         delta_years,
                         tag,
                        ])
print('Done')

Starting


/apps/rapidsai/21.12/lib/python3.8/site-packages/cudf/core/frame.py:3086: FutureWarning: Calling take with a boolean array is deprecated and will be removed in the future.
  warnings.warn(


Done


In [170]:
import time
date1 = "2015/12/31"
date2 = "2016/01/01"
newdate1 = time.strptime(date1, "%Y/%m/%d")
newdate2 = time.strptime(date2, "%Y/%m/%d")
newdate1

time.struct_time(tm_year=2015, tm_mon=12, tm_mday=31, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=365, tm_isdst=-1)

In [184]:
newdate1

time.struct_time(tm_year=2010, tm_mon=6, tm_mday=2, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=153, tm_isdst=-1)

In [153]:
img[8:17]

'21015_0_0'

In [141]:
df6.eid.to_numpy().item()

ValueError: can only convert an array of size 1 to a Python scalar

In [147]:
df6['21003-0.0'].to_numpy().item()

'68'

In [179]:
df6['53-0.0'].to_numpy().item()[0:4]

'2010'